In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st

In [2]:
data = pd.read_csv("AB_test_data.csv")

def purchase_dummies(data):
    if data == False:
        return int("0")
    else:
        return int("1")

data['purchase_TF'] = data['purchase_TF'].apply(purchase_dummies)

In [3]:
data.id.nunique()

55000

In [4]:
data.isnull().sum()

Variant        0
purchase_TF    0
date           0
id             0
dtype: int64

In [5]:
data.head()

,Variant,purchase_TF,date,id
0,A,0,2019-12-26,0x6f9421
1,A,0,2019-08-16,0x59d442
2,A,1,2019-03-18,0x6db8f8
3,A,0,2019-02-13,0x68245d
4,A,0,2019-09-28,0x28566e


## Is B better than A?

In [6]:
# number of samples
n_A = data.query('Variant=="A"').shape[0]
n_B = data.query('Variant=="B"').shape[0]
print(n_A)
print(n_B)

50000
5000


In [7]:
# number of conversion
convert_A = data.query('Variant=="A" & purchase_TF=="1"').shape[0]
convert_B = data.query('Variant=="B" & purchase_TF=="1"').shape[0]
print(convert_A,convert_B)

7603 981


In [8]:
# convert rate
p_A = convert_A / n_A
p_B = convert_B / n_B

print(p_A, p_B)

0.15206 0.1962


In [9]:
from scipy.stats import norm
z_alpha = st.norm.ppf(1-0.05/2)
z_alpha

1.959963984540054

In [10]:
z = (p_B-p_A) / np.sqrt(p_A*(1-p_A)/n_B)
z

8.692151285198767

Because z > z_alpha, we can reject H0: p_A >= p_B and accept that p_A < p_B.

## Solving optimal sample size

In [11]:
from scipy.stats import norm
z_beta = norm.ppf(0.2)
z_alpha_2 = norm.ppf(0.025)

In [12]:
n = ((z_alpha_2+z_beta)**2 * (p_A*(1-p_A)+p_B*(1-p_B)))/(p_A-p_B)**2
n

1154.741666739501

## Conduct the test 10 times

In [13]:
bcr = p_A  # baseline conversion rate
d_hat = p_B-p_A # difference between the groups
print(bcr, d_hat)

0.15206 0.04414000000000001


In [81]:
def create_df(N_concat):
    
    alist=[]
    
    convert_N_A = N_concat.query('Variant=="A" & purchase_TF=="1"').shape[0]
    convert_N_B = N_concat.query('Variant=="B" & purchase_TF=="1"').shape[0]
    p_N_A = convert_N_A / 50000
    p_N_B = convert_N_B / 1155
    
    z_alpha = st.norm.ppf(1-0.05/2)
    z = (p_N_B-p_N_A) / np.sqrt(p_N_A*(1-p_N_A)/1155)
    
    reject_H0 = z > z_alpha 
    
    alist.append(p_N_B)
    alist.append(z)
    alist.append(reject_H0)
    
    return alist
    

In [82]:
adict={}
i=0

while i < 10:
    N_A = data[data["Variant"]=="A"]
    N_B = data[data["Variant"]=="B"].sample(n=1155)
    N_concat = pd.concat([N_A,N_B])
    adict[i]=create_df(N_concat)
    i += 1

In [86]:
df = pd.DataFrame(adict)
df = df.T
df.rename(columns={0:'p_sample', 1:'z_score', 2:'reject_H0'},inplace=True)
df

,p_sample,z_score,reject_H0
0,0.191342,3.71787,True
1,0.186147,3.22621,True
2,0.203463,4.86509,True
3,0.212121,5.68454,True
4,0.199134,4.45537,True
5,0.201732,4.7012,True
6,0.185281,3.14426,True
7,0.192208,3.79982,True
8,0.193074,3.88176,True
9,0.193939,3.96371,True


## Conduct a sequential test for the 10 samples

In [87]:
# upper bound
A = (1-0.2)/0.05

# lower bound
B = 0.2/(1-0.05)

print(A,B)

16.0 0.2105263157894737


In [105]:
samples = data.sample(n=10)
samples

,Variant,purchase_TF,date,id
41947,A,1,2019-09-22,0x574fa5
32468,A,1,2019-07-10,0x13e555
12504,A,0,2020-01-16,0x6c5841
14655,A,0,2019-11-07,0x17490f
7805,A,0,2019-06-09,0x606309
14518,A,0,2019-07-23,0x60c2b9
39347,B,0,2020-01-30,0x3a8c5f
25565,A,0,2019-10-01,0x374b24
14360,A,0,2019-03-16,0x345b51
11635,A,1,2019-10-08,0x5fa8cd


In [ ]:
for index, rows in samples.itterrows():
    
    log_gamma = 0
    count = 0